In [17]:
# Setup & Imports 
import pandas as pd    # import the real pandas first
import numpy as np
import random
from copy import deepcopy

import os, sys
sys.path.insert(0, os.path.abspath('..'))

# Import the GA modules
from ga_rd.algorithm    import run_ga
from ga_rd.selection    import tournament_selection, rank_selection
from ga_rd.crossover    import pmx_crossover, uniform_crossover
from ga_rd.mutation     import swap_mutation, scramble_mutation, inversion_mutation
from ga_rd.Individual_RD import IndividualRD
from ga_rd.Solution_RD  import SolutionRD

print("Imports OK")

Imports OK


In [2]:
# Load and get the participant homes assigned 

df = pd.read_csv("../Data/coordinates_map.csv", index_col="Index")

# define the number of participants
n_participants = 18

# guest-home coords for all diners
participant_homes = df[["Latitude","Longitude"]].values[:n_participants]

# pick exactly the number of hosting houses you need
houses_per_course = int(np.ceil(n_participants / 6))   # if capacity=6
n_houses           = houses_per_course*3 + 1
host_idxs          = random.sample(range(n_participants), n_houses)
house_coords       = participant_homes[host_idxs]

print("participants:", participant_homes.shape)  # (18,2)
print("hosts:       ",    house_coords.shape)   # (10,2)

participants: (18, 2)
hosts:        (10, 2)


In [3]:
# Test one IndividualRD 
ind = IndividualRD(
    participant_homes  = participant_homes,
    house_coords       = house_coords,
    host_idxs         = host_idxs,
    capacity_of_houses = 6,
    a                  = 1.0,
    afterparty_house   = None,   # optional
    crossover_method   = 'uniform',
    crossover_prob     = 0.9,
    mutation_method    = 'swap',
    mutation_prob      = 0.1
)
ind.random_representation()
print("Genome length:", ind.solution.genome.size)
print("Sample genome:", ind.solution.genome)
print("Fitness:", ind.fitness)

Genome length: 64
Sample genome: [ 2  2 -1  0  1  1  0  0  2  1  4  8  7  9  1 16 15  2  5 14 12  6 11 10
 17 13  3  0 12 16  3 13  1  2 15  7  6 10  0 11  4  9  5  8 14 17 10 16
  4  9  7 17 11  1  8 14 13  0 12 15  3  6  2  5]
Fitness: 6.385193951500603


In [4]:
# Check every course's block to see if it contains all participants and no duplicates 
for c in range(ind.solution.n_courses):
    _, block = ind.solution.get_course(c)
    print(f"Course {c} block: {block}")
    print(f"Sorted: {np.sort(block)}")
    print(f"Expected: {np.arange(ind.solution.n_participants)}")
    print("Block length:", len(block))
    print("Expected length:", ind.solution.n_participants + ind.solution.empty_spots)
    print()


Course 0 block: [ 4  8  7  9  1 16 15  2  5 14 12  6 11 10 17 13  3  0]
Sorted: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Block length: 18
Expected length: 18

Course 1 block: [12 16  3 13  1  2 15  7  6 10  0 11  4  9  5  8 14 17]
Sorted: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Block length: 18
Expected length: 18

Course 2 block: [10 16  4  9  7 17 11  1  8 14 13  0 12 15  3  6  2  5]
Sorted: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Block length: 18
Expected length: 18



In [5]:
# Test selection operators 
# build a small population
pop = [
    IndividualRD(
        participant_homes=participant_homes,
        house_coords     =house_coords,
        host_idxs        =host_idxs,
        capacity_of_houses=6,
        a                =1.0
    )
    for _ in range(20)
]
# now randomize each genome
for ind in pop:
    ind.random_representation()

# test the selection mechanisms
t = tournament_selection(pop, k=3)
r = rank_selection(pop)
print("Tournament winner fitness:", t.fitness)
print("Tournament winner genome:", t.solution.genome)
print("Rank‐selection winner  fitness:", r.fitness)
print("Rank‐selection winner  genome:", r.solution.genome)


Tournament winner fitness: 5.934712338580335
Tournament winner genome: [ 0  0  2  1  1  2  1  2  0 -1  3 12 11  7 17  9 14  0 10  4  1 13  6  5
 16  8  2 15  1  3 10 11 14  7  0  8 15 12 16  9  5  6  2 13 17  4 17  1
  5  3 14  2  7  9 12  0 15  4 10  8 11 16  6 13]
Rank‐selection winner  fitness: 5.725752806367674
Rank‐selection winner  genome: [ 2  0 -1  2  1  0  1  0  2  1 10  0  7  4 13  1  8  6 14 12 15  3 16 11
  2 17  5  9 16  7 10 15  2  8  0 13 12  5  3 14 11  6  1  9 17  4  4 16
 11  9  1  0 13  3 14 17 15  7  5  8 10  2 12  6]


In [6]:
# Visualize hosts and verify they’re in their own houses
def house_assignment_table(sol):
    """
    Returns a DataFrame indexed by house number, showing:
    - Course hosted in that house
    - Host ID (participant number)
    - Whether host is present in the house
    - List of house members
    """
    rows = []
    for h in range(sol.n_houses):
        course_idx, members = sol.get_house(h)
        host_id = sol.host_idxs[h]
        host_present = host_id in members
        rows.append({
            "Course hosted": course_idx if course_idx >= 0 else "–",
            "Host ID": host_id,
            "Host present": host_present,
            "House members": ", ".join(str(m) for m in members)
        })
    df = pd.DataFrame(rows, index=[f"House {h}" for h in range(sol.n_houses)])
    return df

sol = pop[0].solution
display(house_assignment_table(sol))

,Course hosted,Host ID,Host present,House members
House 0,1,9,True,"8, 9, 10, 11, 2, 13"
House 1,–,0,False,
House 2,1,1,True,"6, 1, 16, 5, 12, 7"
House 3,2,7,True,"9, 3, 2, 7, 1, 8"
House 4,2,16,True,"13, 11, 16, 15, 4, 17"
House 5,1,15,True,"3, 4, 15, 0, 17, 14"
House 6,2,5,True,"12, 14, 0, 5, 6, 10"
House 7,0,11,True,"13, 6, 11, 0, 9, 16"
House 8,0,2,True,"3, 15, 17, 5, 2, 14"
House 9,0,4,True,"1, 12, 10, 4, 7, 8"


In [7]:
# Profile & Validate All Crossover Variants 
####### NEED TO FIX CROSSOVER METHODS #######
import time
import numpy as np
from IPython.display import display, Markdown

# assuming you already have `pop` from previous cells:
p1, p2 = pop[0], pop[1]

# 2) uniform_crossover at the SolutionRD level
from ga_rd.crossover import uniform_crossover
t0 = time.perf_counter()
u1, u2 = uniform_crossover(p1.solution, p2.solution, verbose=True, max_retries=10)
dt_uniform = time.perf_counter() - t0

# 3) pmx_crossover at the SolutionRD level
from ga_rd.crossover import pmx_crossover
t0 = time.perf_counter()
o1, o2 = pmx_crossover(p1.solution, p2.solution, verbose=True, max_retries=10)
dt_onept = time.perf_counter() - t0

# Helper to check validity
def check(sol):
    return sol.check_validity_of_genome(verbose=True)

results = [
    ("IndividualRD.crossover", dt_ind, child1.solution, child2.solution),
    ("uniform_crossover",      dt_uniform, u1,      u2     ),
    ("pmx_crossover",    dt_onept,    o1,      o2     ),
    # ("pmx_crossover",        dt_pmx,      x1,      x2     ),
]

for name, dt, cA, cB in results:
    display(Markdown(f"### {name} (took {dt:.4f}s)"))
    validA = check(cA)
    validB = check(cB)
    display(Markdown(f"- Child A valid? **{validA}**; fitness = {cA.fitness:.3f}"))
    display(Markdown(f"- Child B valid? **{validB}**; fitness = {cB.fitness:.3f}"))
    # if you want to see their genomes:
    # print("  A genome:", cA.genome)
    # print("  B genome:", cB.genome)


[uniform] attempt 1
[uniform] attempt 2
[uniform] attempt 3
[uniform] attempt 4
[uniform] attempt 5
[uniform] attempt 6
[uniform] attempt 7
[uniform] attempt 8
[uniform] attempt 9
[uniform] attempt 10
[uniform] retries exhausted; falling back per child
[PMX] attempt 1, segment=[0:10], crossover points=[0:2]


KeyboardInterrupt: 

In [15]:
# Test mutation operators 
###### NEED TO FIX MUTATION METHODS ######
# I need to perform the mutation if it is required insted of falling back to the original genome
m1 = swap_mutation(pop[2], verbose=True)
print("before mutation fitness:", pop[2].fitness, "after swap_mutation fitness:", m1.fitness)
print("Genome before mutation:", pop[2].solution.genome, "\nGenome after swap_mutation:\n", m1.solution.genome)

m2 = scramble_mutation(pop[2], verbose=True)
print("before mutation fitness:", pop[2].fitness, "after scramble_mutation fitness:", m2.fitness)
print("Genome before mutation:", pop[2].solution.genome, "\nGenome after scramble_mutation:\n", m2.solution.genome)
m3 = inversion_mutation(pop[2], verbose=True)
print("before mutation fitness:", pop[2].fitness, "after inversion_mutation fitness:", m3.fitness)
print("Genome before mutation:\n", pop[2].solution.genome, "\nGenome after inversion_mutation:\n", m3.solution.genome)


[swap_mutation] All retries failed, returning original
before mutation fitness: 7.504633402903922 after swap_mutation fitness: 7.504633402903922
Genome before mutation: [-1  1  0  2  0  0  2  1  2  1 13 11 12  1  2  7  0 10  9 16  3  8 15  5
  4 17 14  6 13  5  6  0 17  8  2 11  9  7 15  3 12 16  4 14 10  1  7  9
 12 13 14 11 10 17  0  1  5  4 15  8  6 16  3  2] 
Genome after swap_mutation:
 [-1  1  0  2  0  0  2  1  2  1 13 11 12  1  2  7  0 10  9 16  3  8 15  5
  4 17 14  6 13  5  6  0 17  8  2 11  9  7 15  3 12 16  4 14 10  1  7  9
 12 13 14 11 10 17  0  1  5  4 15  8  6 16  3  2]
[scramble_mutation] Success on attempt 3: scrambled 1:5
before mutation fitness: 7.504633402903922 after scramble_mutation fitness: 7.504633402903922
Genome before mutation: [-1  1  0  2  0  0  2  1  2  1 13 11 12  1  2  7  0 10  9 16  3  8 15  5
  4 17 14  6 13  5  6  0 17  8  2 11  9  7 15  3 12 16  4 14 10  1  7  9
 12 13 14 11 10 17  0  1  5  4 15  8  6 16  3  2] 
Genome after scramble_mutation:
 [-1  

In [9]:
# Run a small GA and inspect the best solution
population, log = run_ga(
    pop_size          = 20,
    participant_homes = participant_homes,
    house_coords      = house_coords,
    host_idxs         = host_idxs,
    capacity          = 6,
    a                 = 1.0,
    generations       = 5,
    selection_method  = 'tournament',
    tournament_size   = 3,
    afterparty_house  = host_idxs[0]
)

best = min(population, key=lambda ind: ind.fitness)
print("Best fitness:", best.fitness)
print("Best genome: ", best.solution.genome)

Best fitness: 6.102343877330302
Best genome:  [-1  2  2  1  1  0  0  1  0  2  7 11  1 17 15 14  0  5 13  6 16  4  9 12
  8 10  2  3 17  7  6  4  2  5  8 13 12 15 16 14  1  9  3  0 10 11 16  2
  0 14  6 12 11  1 13 10  8  7  5  3  9 15 17  4]


In [10]:
# Define run_ga_verbose ===
import time
import numpy as np

def run_ga_verbose(
    pop_size: int,
    participant_homes: np.ndarray,
    house_coords:      np.ndarray,
    host_idxs:         np.ndarray,
    capacity_of_houses:int,
    a:                 float,
    generations:       int,
    selection_method:  str   = 'tournament',
    tournament_size:   int   = 3,
    afterparty_house:  int   = None
):
    """
    Runs `generations` of the RD‐GA, printing stats each generation.

    Args:
        pop_size:           number of individuals
        participant_homes:  array, shape=(n_participants,2)
        house_coords:       array, shape=(n_houses,2)
        capacity_of_houses: seats per house
        a:                  distance vs mixing weight
        generations:        how many generations to run
        selection_method:   'tournament' or 'rank'
        tournament_size:    k for tournament
        afterparty_house:   optional index of after‐party house

    Returns:
        final population list and a list of per‐gen stats dicts
    """
    # 1) initialize random population
    pop = [
        IndividualRD(
            participant_homes=participant_homes,
            house_coords=house_coords,
            host_idxs=host_idxs,
            capacity_of_houses=capacity_of_houses,
            a=a,
            afterparty_house=afterparty_house
        )
        for _ in range(pop_size)
    ]
    for ind in pop:
        ind.random_representation()

    stats = []
    start_all = time.perf_counter()

    # 2) generational loop
    for gen in range(1, generations+1):
        t0 = time.perf_counter()
        new_pop = []

        while len(new_pop) < pop_size:
            # parent selection
            if selection_method == 'tournament':
                p1 = tournament_selection(pop, tournament_size)
                p2 = tournament_selection(pop, tournament_size)
            else:
                p1 = rank_selection(pop)
                p2 = rank_selection(pop)

            # crossover
            c1, c2 = p1.crossover(p2)

            # mutation
            m1 = c1.mutation()
            m2 = c2.mutation()

            new_pop.extend([m1, m2])

        # trim in case we went over
        pop = new_pop[:pop_size]

        # compute stats
        fitnesses = np.array([ind.fitness for ind in pop])
        best_idx  = int(np.argmin(fitnesses))
        worst_idx = int(np.argmax(fitnesses))
        best_f    = fitnesses[best_idx]
        avg_f     = fitnesses.mean()
        worst_f   = fitnesses[worst_idx]
        best_genome = pop[best_idx].solution.genome.copy()

        dt = time.perf_counter() - t0
        print(
            f"Gen {gen:2d} — "
            f"best={best_f:.3f}, avg={avg_f:.3f}, worst={worst_f:.3f} "
            f"({dt:.2f}s)"
        )

        stats.append({
            'gen':       gen,
            'best':      best_f,
            'average':   avg_f,
            'worst':     worst_f,
            'best_genome': best_genome
        })

    total_t = time.perf_counter() - start_all
    print(f"\nFinished {generations} gens in {total_t:.2f}s")
    return pop, stats

In [11]:
# === Cell Y: actually run it ===
final_pop, run_stats = run_ga_verbose(
    pop_size=20,
    participant_homes=participant_homes,
    house_coords=house_coords,
    host_idxs=host_idxs,
    capacity_of_houses=6,
    a=1.0,
    generations=100,
    selection_method='tournament',
    tournament_size=3,
    afterparty_house=host_idxs[0]
)

# grab the best at the end
best = min(final_pop, key=lambda ind: ind.fitness)
print("\nFINAL BEST:", best.fitness, "genome:", best.solution.genome)


Gen  1 — best=6.087, avg=6.905, worst=7.630 (0.61s)
Gen  2 — best=6.087, avg=6.139, worst=6.603 (0.33s)
Gen  3 — best=6.087, avg=6.087, worst=6.087 (0.13s)
Gen  4 — best=6.087, avg=6.111, worst=6.566 (0.14s)
Gen  5 — best=6.087, avg=6.111, worst=6.566 (0.13s)
Gen  6 — best=6.087, avg=6.087, worst=6.087 (0.13s)
Gen  7 — best=6.087, avg=6.097, worst=6.269 (0.15s)
Gen  8 — best=6.087, avg=6.095, worst=6.231 (0.14s)
Gen  9 — best=6.087, avg=6.087, worst=6.087 (0.15s)
Gen 10 — best=6.087, avg=6.100, worst=6.332 (0.13s)
Gen 11 — best=6.087, avg=6.092, worst=6.169 (0.13s)
Gen 12 — best=6.087, avg=6.087, worst=6.087 (0.13s)
Gen 13 — best=6.087, avg=6.090, worst=6.144 (0.13s)
Gen 14 — best=6.087, avg=6.087, worst=6.087 (0.13s)
Gen 15 — best=6.087, avg=6.087, worst=6.087 (0.13s)
Gen 16 — best=6.087, avg=6.087, worst=6.087 (0.12s)
Gen 17 — best=6.087, avg=6.087, worst=6.087 (0.13s)
Gen 18 — best=6.087, avg=6.087, worst=6.087 (0.13s)
Gen 19 — best=6.087, avg=6.087, worst=6.087 (0.14s)
Gen 20 — bes

In [22]:
# 1. Create one “base” individual and randomize it
ind1 = IndividualRD(
    participant_homes=participant_homes,
    house_coords=house_coords,
    host_idxs=host_idxs,
    capacity_of_houses=6,
    a=1.0
)
ind1.random_representation()

# 2. Make a “re‐ordered” twin that only permutes within-house guest lists
ind2 = deepcopy(ind1)
sol = ind2.solution
# for house 0 only, shuffle the members in place:
ci, start = sol.get_house_position(0)
if ci >= 0:
    block = sol.genome[start : start + sol.capacity_of_houses]
    # remove -1’s, shuffle the rest, re-pad with -1
    guests = [g for g in block if g >= 0]
    import random
    random.shuffle(guests)
    newblock = guests + [-1] * (len(block) - len(guests))
    sol.genome[start : start + sol.capacity_of_houses] = newblock

# 3. Compare raw genomes vs. semantic keys
print("Raw genomes identical?", (ind1.solution.genome == ind2.solution.genome).all())
print("Semantic keys identical?", ind1.semantic_key() == ind2.semantic_key())


Raw genomes identical? False
Semantic keys identical? True
